Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.
    
> steps 3001, reg_lambda .01, Test accuracy: 91.0%<br>
> steps 3001, reg_lambda .001, Test accuracy: 91.3%<br>
> steps 3001, reg_lambda .0001, Test accuracy: 88.9%<br>
> steps 5001, reg_lambda .001, Test accuracy: **93.15%**<br>
> steps 6001, reg_lambda .001, Test accuracy: **91.17%**<br>
> steps 7001, reg_lambda .001, Test accuracy: **93.56%**<br>
> steps 8001, reg_lambda .001, Test accuracy: **91.30%**<br>
> steps 9001, reg_lambda .001, Test accuracy: **91.19%**<br>

---

** Helper functions **

In [6]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape, constant=0.1):
  initial = tf.constant(constant, shape=shape)
  return tf.Variable(initial)

**Set regularization strength**

In [7]:
reg_lambda = 0.001 # regularization strength

**Build graph with 1-hidden layer nn, RELU, L2 reg**

In [8]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  #biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  biases1 = bias_variable([num_hidden_nodes])  ## init w/ positive bias

  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  #biases2 = tf.Variable(tf.zeros([num_labels]))
  biases2 = bias_variable([num_labels])  ## init w/ positive bias
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  a1 = tf.nn.relu(logits1)
    
  logits2 = tf.matmul(a1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
  # Add the regularization term to the loss.
  loss += reg_lambda * 0.5 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_a1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_a1, weights2) + biases2)
  test_a1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_a1, weights2) + biases2)

**Run the model**

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, lz, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step {:d}: {:.4f}".format(step, lz)
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
      print '------------------------'
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 530.8057
Minibatch accuracy: 7.8%
Validation accuracy: 29.1%
------------------------
Minibatch loss at step 500: 156.8192
Minibatch accuracy: 73.4%
Validation accuracy: 77.9%
------------------------
Minibatch loss at step 1000: 99.0400
Minibatch accuracy: 77.3%
Validation accuracy: 81.0%
------------------------
Minibatch loss at step 1500: 80.1773
Minibatch accuracy: 76.6%
Validation accuracy: 79.6%
------------------------
Minibatch loss at step 2000: 57.5673
Minibatch accuracy: 85.9%
Validation accuracy: 82.6%
------------------------
Minibatch loss at step 2500: 44.8506
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
------------------------
Minibatch loss at step 3000: 34.7512
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
------------------------
Test accuracy: 91.05%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

<div style="text-align:center; margin:1.5em;">**num_steps = 31**</div>

| batch size | num steps | L2 reg | Test accuracy | Learning rate |
|------------|-----------|--------|---------------|---------------|
|128 |31| none     | 82.3% |.5|
|128 |31| .01      | 84.4% |"|
|128 |31| .001     | 82.9% |"|
|6000|31| .01      | 87.4% |"|
|128|301| .01      | 86.85% | .9 |
|128|31| .01      | 81.57% | .9 |
|128|31| .001      | 84.62% | .2 |

---

In [35]:
reg_L2 = 0.001
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  b = tf.Variable(tf.constant(0., shape=[num_hidden_nodes]))

  w2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  b2 = tf.Variable(tf.constant(1., shape=[num_labels]))
  
  # Training computation.
  h = tf.nn.relu(tf.matmul(tf_train_dataset, w) + b)
    
  h2 = tf.matmul(h, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(h2, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = tf.nn.l2_loss(w) + tf.nn.l2_loss(w2)
  # Add the regularization term to the loss.
  loss += reg_L2 * 0.5 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(h2)
  h_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, w) + b)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h_valid, w2) + b2)
  h_test = tf.nn.relu(tf.matmul(tf_test_dataset, w) + b)
  test_prediction = tf.nn.softmax(
    tf.matmul(h_test, w2) + b2)

In [36]:
num_steps = 101

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, lz, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 20 == 0):
      print "Minibatch loss at step {:d}: {:.4f}".format(step, lz)
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
      print '------------------------'
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 454.2202
Minibatch accuracy: 7.0%
Validation accuracy: 29.3%
------------------------
Minibatch loss at step 20: 258.2012
Minibatch accuracy: 67.2%
Validation accuracy: 72.3%
------------------------
Minibatch loss at step 40: 235.5361
Minibatch accuracy: 70.3%
Validation accuracy: 76.1%
------------------------
Minibatch loss at step 60: 194.7704
Minibatch accuracy: 79.7%
Validation accuracy: 76.2%
------------------------
Minibatch loss at step 80: 201.8122
Minibatch accuracy: 73.4%
Validation accuracy: 77.8%
------------------------
Minibatch loss at step 100: 200.6046
Minibatch accuracy: 71.1%
Validation accuracy: 77.0%
------------------------
Test accuracy: 84.62%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

> ** Drop input layer **<br>
> num_steps 31, no L2 reg: Test accuracy: **81.3%**<br>
> num_steps 31, reg_lambda .01: Test accuracy: **84.0%**<br>
> num_steps 31, reg_lambda .001: Test accuracy: **82.9%**<br>
> num_steps 3001, reg_lambda .01: Test accuracy: **90.2%**<br>
> num_steps 3001, no L2 reg: Test accuracy: **86.9%**<br>
> num_steps 3001, reg_lambda .01 w2, keep_prob 0.4: Test accuracy: **85.6%**<br>
> num_steps 3001, reg_lambda .001, keep_prob 0.5: Test accuracy **89.8%**<br>
> num_steps 3001, reg_lambda .001, keep_prob 0.7: Test accuracy **90.5%**<br>
> num_steps 3001, reg_lambda .01, keep_prob 0.9: Test accuracy **90.8%**<br>
> num_steps 3001, reg_lambda .001, keep_prob 0.9: Test accuracy **91.1%**<br>
> num_steps 3001, reg_lambda .001 w2, keep_prob 0.9: Test accuracy **88.6%**<br>

> ** Drop hidden layer **<br>
> num_steps 3001, reg_lambda .001, keep_prob 0.5: Test accuracy **92.0%**<br>
> num_steps 5001, reg_lambda .001, keep_prob 0.5: Test accuracy **91.68%**<br>
> num_steps 3001, reg_lambda .001, keep_prob 0.8: Test accuracy **90.80%**<br>
> num_steps 4001, reg_lambda .001, keep_prob 0.65: Test accuracy **90.19%**<br>

---

In [38]:
reg_L2 = 0.001
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  b = tf.Variable(tf.zeros([num_hidden_nodes]))

  w2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  b2 = tf.Variable(tf.constant(.1, shape=[num_labels]))
  
  # Training computation.
  h = tf.nn.relu(tf.matmul(tf_train_dataset, w) + b)
  h2 = tf.matmul(h, w2) + b2

  # Dropout
  keep_prob = tf.placeholder(tf.float32)
  h2_drop = tf.nn.dropout(h2, keep_prob)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(h2_drop, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = tf.nn.l2_loss(w) + tf.nn.l2_loss(w2)
  # Add the regularization term to the loss.
  loss += reg_L2 * 0.5 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.25).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(h2)
  h_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, w) + b)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h_valid, w2) + b2)
  h_test = tf.nn.relu(tf.matmul(tf_test_dataset, w) + b)
  test_prediction = tf.nn.softmax(
    tf.matmul(h_test, w2) + b2)

In [39]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, \
                keep_prob: 0.65}
    ## training use dropout, evaluation/predictions no dropout
    _, lz, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
 
    if (step % 750 == 0):
      print "Minibatch loss at step {:d}: {:.4f}".format(step, lz)
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
      print '------------------------'
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 565.2769
Minibatch accuracy: 4.7%
Validation accuracy: 18.4%
------------------------
Minibatch loss at step 750: 140.5265
Minibatch accuracy: 77.3%
Validation accuracy: 79.7%
------------------------
Minibatch loss at step 1500: 123.0938
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
------------------------
Minibatch loss at step 2250: 90.5980
Minibatch accuracy: 82.8%
Validation accuracy: 83.4%
------------------------
Minibatch loss at step 3000: 74.2476
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
------------------------
Minibatch loss at step 3750: 61.1324
Minibatch accuracy: 80.5%
Validation accuracy: 83.1%
------------------------
Test accuracy: 90.19%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers. eg, 1024-300-50

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

> **Drop input layer**<br>
> steps 3001, reg L2 .001, keep_prob 0.95, decay (.5, 31, .96), Test accuracy: **90.4%**<br>
> steps 3001, reg L2 .001, keep_prob 0.9, decay (.5, 31, .96, stair), Test accuracy: **90.35**<br>
> steps 6001, reg L2 .01, keep_prob 0.9, decay (.5, 31, .96, stair), Test accuracy: **92.88**<br>
> steps 6001, reg L2 .01, keep_prob 0.9, decay (.5, 100, .96, stair), Test accuracy: **92.54**<br>

> **Drop hidden layer**<br>
> steps 3001, reg L2 .001, keep_prob 0.9, decay (.5, 100, .96), Test accuracy: **91.05%**<br>
> steps 3001, reg L2 .001, keep_prob 0.9, decay (.5, 31, .96), Test accuracy: **90.18%**<br>
> steps 5001, reg L2 .001, keep_prob 0.5, decay (.5, 100, .96), Test accuracy: **90.90%**<br>
> steps 6001, reg L2 .001, keep_prob 0.7, decay (.5, 100, .96, stair), Test accuracy: **91.83%**<br>
> steps 6001, reg L2 .001, keep_prob 0.9, decay (.5, 60, .96, stair), Test accuracy: **91.38%**<br>

> **Drop hidden layer, biases non-zero**<br>
> steps 3001, reg L2 .001, keep_prob 0.5, decay (.5, 50, .96), Test accuracy: ** %**<br>

 ---


Learning rate decay
------------------

In [ ]:
reg_L2 = 0.001
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  b = tf.Variable(tf.zeros([num_hidden_nodes]))

  w2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h = tf.nn.relu(tf.matmul(tf_train_dataset, w) + b)

  # Dropout
  keep_prob = tf.Variable(0.9, trainable=False)
  #keep_prob = tf.placeholder(tf.float32)
  
  h2 = tf.matmul(h, w2) + b2
  h2_drop = tf.nn.dropout(h2, keep_prob)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(h2_drop, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  reg_w = tf.nn.l2_loss(w) + tf.nn.l2_loss(b)
  reg_w2 = tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2)
  regularizers = reg_w + reg_w2
  # Add the regularization term to the loss.
  loss += reg_L2 * 0.5 * regularizers
  
  # Optimizer.
  ## with learning rate decay
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 10, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(h2)
  h_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, w) + b)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h_valid, w2) + b2)
  h_test = tf.nn.relu(tf.matmul(tf_test_dataset, w) + b)
  test_prediction = tf.nn.softmax(
    tf.matmul(h_test, w2) + b2)

In [216]:
num_steps = 101

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, lz, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print global_step.eval(), learning_rate.eval()
      print "Minibatch loss at step {:d}: {:.4f}".format(step, lz)
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.2f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
      print '------------------------'
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
1 0.5
Minibatch loss at step 0: 468.0904
Minibatch accuracy: 14.06%
Validation accuracy: 35.00%
------------------------
51 0.407686
Minibatch loss at step 50: 197.0026
Minibatch accuracy: 74.22%
Validation accuracy: 75.42%
------------------------
101 0.332416
Minibatch loss at step 100: 201.6727
Minibatch accuracy: 67.19%
Validation accuracy: 75.34%
------------------------
Test accuracy: 82.51%


---

Add Hidden Layers
--------------

** 2-hidden layers, 1024 x 50 **
> steps 3001, L2 reg .001, keep 0.9, decay (.5, 60, .96, stair), Test acc: **10.00**%<br>
> steps 3001, L2 reg .001 (w's), keep 0.7, decay none (.01), Test acc: **13.93**%<br>

** 2-hidden layers, 1024 x 300 **
> steps 3001, L2 reg .001 (w's), keep 0.5, decay (.05, 60, .96, stair), Test acc: **12.14**%<br>
> steps 6001, L2 reg .001 (w's), keep 0.9, decay (.09, 60, .96, stair), Test acc: **10.97**%<br>
> steps 3001, L2 reg none, keep 1., decay (.05, 128, .96, stair), Test acc: **51.81**%<br>
> steps 3001, L2 reg none, keep 1., decay (.05, 128, .96), Test acc: **43.08**%<br>
> steps 3001, L2 reg none, keep 1., decay (.025, 60, .96, stair), Test acc: **56.94**%<br>
> steps 100, L2 reg none, keep 1., decay (.025, 60, .96, stair), Test acc: **65.57**%<br>
> steps 1001, L2 reg none, keep 1., decay (.025, 20, .96, stair), Test acc: **49.06%**%<br>

** 2-hidden layers, 1024 tanh x 300 relu**
> steps 3001, L2 reg .001, keep 1., decay none (.5), Test acc: **61.65**%<br>
> steps 301, L2 reg .001, keep 1., decay (.3, 10, .95), Test acc: **66.72**%<br>


---

** 2-hidden layers **

In [64]:
reg_L2 = 0.001
batch_size = 128
num_hidden_1 = 1024
num_hidden_2 = 300

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_1]))
  b = tf.Variable(tf.zeros([num_hidden_1]))

  w2 = tf.Variable(
    tf.truncated_normal([num_hidden_1, num_hidden_2]))
  b2 = tf.Variable(tf.zeros([num_hidden_2]))
    
  w3 = tf.Variable(
    tf.truncated_normal([num_hidden_2, num_labels]))
  b3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h = tf.tanh(tf.matmul(tf_train_dataset, w) + b)
  h2 = tf.nn.relu(tf.matmul(h, w2) + b2)

  # Dropout
  keep_prob = tf.placeholder(tf.float32)
  h2_drop = tf.nn.dropout(h2, keep_prob)

  h3 = tf.matmul(h2, w3) + b3
  h3_drop = tf.nn.dropout(tf.matmul(h2_drop, w3) + b3, keep_prob)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(h3_drop, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = tf.nn.l2_loss(w) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3)
  # Add the regularization term to the loss.
  loss += reg_L2 * 0.5 * regularizers
  
  # Optimizer.
  ##optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  ## Optimizer with learning rate decay
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.01, global_step, 20, 0.9, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(h3)
  h_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, w) + b)
  h2_valid = tf.nn.relu(tf.matmul(h_valid, w2) + b2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h2_valid, w3) + b3)
  h_test = tf.nn.relu(tf.matmul(tf_test_dataset, w) + b)
  h2_test = tf.nn.relu(tf.matmul(h_test, w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(h2_test, w3) + b3)

** Run it **

In [ ]:
num_steps = 401

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, \
                keep_prob: .5}
    _, lz, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print global_step.eval(), learning_rate.eval()  ## show learning decay
      print "Minibatch loss at step {:d}: {:.4f}".format(step, lz)
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.2f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
      print '------------------------'
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

---
3-hidden layers
---------------

** 3-hidden layers, 1024 x 300 x 50**
> steps 1001, L2 reg none, keep 1., decay (.005, 20, .96), Test acc: **10.44**%<br>
> steps 3001, L2 reg none, keep 1., decay (.001, 20, .95, stair), Test acc: **10.97**%<br>
> steps 3001, L2 reg none, keep 1., decay (.002, 60, .96, stair), Test acc: **10.40**%<br>
> steps 3001, L2 reg none, keep 1., decay none (.005), Test acc: **10.01**%<br>

** 3-hidden layers, 1024 tanh x 300 tanh x 50 relu**
> steps 3001, L2 reg .001, keep 1., decay (.5, 60, .96), Test acc: **48.84**%<br>
> steps 3001, L2 reg .001, keep 1., decay (.5, 600, .96), Test acc: **54.53**%<br>
> steps 3001, L2 reg .0001, keep .5, decay (.04, 100, .96), Test acc: **41.47**%<br>

** 3-hidden layers, 1024 tanh x 300 tanh x 50 tanh, batch 64**
> steps 6001, L2 reg .001, keep .5, decay (.05, 100, .96), Test acc: **72.10**%<br>
> steps 6001, L2 reg .001, keep .5, decay (.01, 200, .96), Test acc: **66.16**%<br>
> steps 3001, L2 reg .001, keep .5, decay (.03, 200, .96), Test acc: **67.59**%<br>
> steps 6001, L2 reg .001, keep .9, decay (.04, 200, .96), Test acc: **79.26**%<br>
> steps 6001, L2 reg .01, keep .9, decay (.045, 150, .96), Test acc: **82.97**%<br>

** 3-hidden layers, 1024 tanh x 300 tanh x 50 tanh, batch 128**
> steps 6001, L2 reg .001, keep .8, decay (.045, 150, .96), Test acc: **78.34**%<br>


---

In [72]:
reg_L2 = 0.01
batch_size = 64
num_hidden_1 = 1024
num_hidden_2 = 300
num_hidden_3 = 50

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_1]))
  b = bias_variable([num_hidden_1], .1)

  w2 = tf.Variable(
    tf.truncated_normal([num_hidden_1, num_hidden_2]))
  b2 = bias_variable([num_hidden_2])
    
  w3 = tf.Variable(
    tf.truncated_normal([num_hidden_2, num_hidden_3]))
  b3 = bias_variable([num_hidden_3])
    
  w4 = tf.Variable(
    tf.truncated_normal([num_hidden_3, num_labels]))
  b4 = bias_variable([num_labels], .1)
  
  # Training computation.
  h = tf.tanh(tf.matmul(tf_train_dataset, w) + b)
  h2 = tf.tanh(tf.matmul(h, w2) + b2)

  # Dropout
  keep_prob = tf.placeholder(tf.float32)
  h_drop = tf.nn.dropout(h, keep_prob)
  h2_drop = tf.tanh(tf.matmul(h_drop, w2) + b2)
  h3 = tf.tanh(tf.matmul(h2, w3) + b3)
  h3_drop = tf.nn.dropout(tf.tanh(tf.matmul(h2_drop, w3) + b3), keep_prob)
  h4 = tf.matmul(h3, w4) + b4
  h4_drop = tf.matmul(h3_drop, w4) + b4
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(h4_drop, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = tf.nn.l2_loss(w) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3)
  # Add the regularization term to the loss.
  loss += reg_L2 * 0.5 * regularizers
  
  # Optimizer.
  ##optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
  ## Optimizer with learning rate decay
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.045, global_step, 150, 0.96, staircase=False)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)  
  ##optimizer = tf.train.GradientDescentOptimizer(.01).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(h4)
  h_valid = tf.tanh(tf.matmul(tf_valid_dataset, w) + b)
  h2_valid = tf.tanh(tf.matmul(h_valid, w2) + b2)
  h3_valid = tf.tanh(tf.matmul(h2_valid, w3) + b3)
  valid_prediction = tf.nn.softmax(
    tf.matmul(h3_valid, w4) + b4)
  h_test = tf.tanh(tf.matmul(tf_test_dataset, w) + b)
  h2_test = tf.tanh(tf.matmul(h_test, w2) + b2)
  h3_test = tf.tanh(tf.matmul(h2_test, w3) + b3)
  test_prediction = tf.nn.softmax(
    tf.matmul(h3_test, w4) + b4)

In [73]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized\n==============="
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, \
                keep_prob: .9}
    _, lz, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 600 == 0):
      print global_step.eval(), learning_rate.eval()  ## show learning decay
      print "Minibatch loss at step {:d}: {:.4f}".format(step, lz)
      print "Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.2f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
      print '------------------------'
  print "Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
1 0.0449878
Minibatch loss at step 0: 2182.4053
Minibatch accuracy: 12.50%
Validation accuracy: 10.59%
------------------------
601 0.0382102
Minibatch loss at step 600: 1697.3685
Minibatch accuracy: 62.50%
Validation accuracy: 59.10%
------------------------
1201 0.0324537
Minibatch loss at step 1200: 1373.6836
Minibatch accuracy: 62.50%
Validation accuracy: 65.87%
------------------------
1801 0.0275644
Minibatch loss at step 1800: 1147.5002
Minibatch accuracy: 70.31%
Validation accuracy: 69.12%
------------------------
2401 0.0234118
Minibatch loss at step 2400: 985.4017
Minibatch accuracy: 62.50%
Validation accuracy: 70.65%
------------------------
3001 0.0198847
Minibatch loss at step 3000: 865.4965
Minibatch accuracy: 78.12%
Validation accuracy: 72.94%
------------------------
3601 0.016889
Minibatch loss at step 3600: 775.1639
Minibatch accuracy: 68.75%
Validation accuracy: 73.45%
------------------------
4201 0.0143446
Minibatch loss at step 4200: 705.9191
Minibatch